In [ ]:
import tensorflow.keras 
from keras.layers import Conv2D,Dense,MaxPooling2D,Dropout,Input,Flatten
from keras.models import Sequential
from keras.applications import vgg16

In [ ]:
import numpy as np
import os 
from keras.preprocessing.image import  ImageDataGenerator

In [ ]:
conv_base=vgg16.VGG16(weights='imagenet',include_top=False,input_shape=(150,150,3)4)

58900480/58889256 [==============================] - 0s 0us/step


In [18]:
# creat my model
base_dir='/base_dir'
train_dir=os.path.join(base_dir,'train')
validatin_dir=os.path.join(base_dir,'validation')
datagen=ImageDataGenerator(rescale=1./255)
batch_size=20

In [9]:
def extract_feature(directory,sample_count):
  features=np.zeros(shape=(sample_count,4,4,512))
  labels=np.zeros(shape=(sample_count))

  generator=datagen.flow_from_directory(directory,target_size=(150,150),batch_size=batch_size,
                                        class_mode='binary')
  i=0
  for input_batch , labels_batch in generator:
    if input_batch.shape[0]==3:
      continue
    
    feature_batch=conv_base.predict(input_batch)
    features[i* batch_size :(i+1)*batch_size]=feature_batch
    labels[i* batch_size :(i+1)*batch_size]=labels_batch

    i+=1
    if i*batch_size >= sample_count:
      break
  return features,labels 


In [16]:
feature_train, label_train=extract_feature(train_dir,2000)

Found 1763 images belonging to 3 classes.


In [19]:
feature_validation,label_validation=extract_feature(validatin_dir,1000)

Found 1000 images belonging to 3 classes.


In [20]:
feature_train=np.reshape(feature_train,(2000,4*4*512))
feature_validation=np.reshape(feature_validation,(1000,4*4*512))

In [37]:
# create model
from keras.optimizer_v1 import  RMSprop
model=Sequential()
model.add(Dense(256,activation='relu',input_dim=4*4*512,kernel_initializer='uniform'))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='RMSprop',metrics=['acc'])

In [38]:
history=model.fit(feature_train,label_train,epochs=30,batch_size=20,validation_data=(feature_validation,label_validation))

Epoch 1/30
100/100 [==============================] - 5s 37ms/step - loss: -23307.9883 - acc: 0.5630 - val_loss: -101238.7500 - val_acc: 0.5000
Epoch 2/30
100/100 [==============================] - 4s 35ms/step - loss: -322748.5000 - acc: 0.5665 - val_loss: -800005.1875 - val_acc: 0.5000
Epoch 3/30
100/100 [==============================] - 3s 35ms/step - loss: -1525271.3750 - acc: 0.5665 - val_loss: -2960982.0000 - val_acc: 0.5000
Epoch 4/30
100/100 [==============================] - 4s 37ms/step - loss: -4383817.5000 - acc: 0.5665 - val_loss: -7604181.0000 - val_acc: 0.5000
Epoch 5/30
100/100 [==============================] - 4s 39ms/step - loss: -9969296.0000 - acc: 0.5665 - val_loss: -16253118.0000 - val_acc: 0.5000
Epoch 6/30
100/100 [==============================] - 4s 41ms/step - loss: -20089066.0000 - acc: 0.5665 - val_loss: -30700958.0000 - val_acc: 0.5000
Epoch 7/30
100/100 [==============================] - 4s 40ms/step - loss: -36054748.0000 - acc: 0.5665 - val_loss: -527